In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import librosa
import os

2025-04-02 13:05:53.456208: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-02 13:05:53.467450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743620753.480828   42623 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743620753.484775   42623 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743620753.494896   42623 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Built with CUDA:", tf.test.is_built_with_cuda())

TensorFlow version: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: True


In [2]:
# Load YAMNet model
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')


I0000 00:00:1743620764.018782   42623 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9763 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:65:00.0, compute capability: 8.9


In [3]:
# Function to load and preprocess audio for YAMNet
def preprocess_audio_for_yamnet(file_path):
    # YAMNet expects 16 kHz mono audio
    audio, sr = librosa.load(file_path, sr=16000, mono=True)
    return audio, sr


In [4]:
# Test YAMNet on a sample file
base_dir = "/data/birdclef/birdclef-2025"
sample_file = os.path.join(base_dir, "train_soundscapes", os.listdir(os.path.join(base_dir, "train_soundscapes"))[0])

print(f"Testing YAMNet on: {sample_file}")
audio, sr = preprocess_audio_for_yamnet(sample_file)


Testing YAMNet on: /data/birdclef/birdclef-2025/train_soundscapes/H63_20230429_222500.ogg


In [8]:
# Run the model on the waveform
scores, embeddings, log_mel_spectrogram = yamnet_model(audio)

# Print the shape of the embeddings
print(f"Embeddings shape: {embeddings.shape}")
print(f"Number of segments: {embeddings.shape[0]}")
print(f"Embedding dimension: {embeddings.shape[1]}")

E0000 00:00:1743621058.115200   42790 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2025-04-02 13:10:58.115648: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at conv_ops_impl.h:1204 : INVALID_ARGUMENT: No DNN in stream executor.
2025-04-02 13:10:58.115675: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: No DNN in stream executor.
	 [[{{node yamnet_frames/layer1/conv/Conv2D}}]]


InvalidArgumentError: Graph execution error:

Detected at node yamnet_frames/layer1/conv/Conv2D defined at (most recent call last):
<stack traces unavailable>
No DNN in stream executor.
	 [[{{node yamnet_frames/layer1/conv/Conv2D}}]] [Op:__inference_restored_function_body_15206]